# [Projet 6: Catégorisez automatiquement des questions](https://openclassrooms.com/fr/projects/categorisez-automatiquement-des-questions)
(parcours data: [here](https://openclassrooms.com/paths/63-data-scientist))

## Intelligence

Resources:
- [tutorial on kaggle](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words) (I did this basically on my own, but look at the features with scikit-learn section)

### Imports

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'Documents/GitHub/OCDataSciencePath/Project6/Work/')    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [3]:
from Pkg.helper import avg_jacard # only works if correct working dirrectory

### Data

In [4]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToDataDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/'    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))

In [11]:
filename = 'QueryResults_10k_forLearning.npz'

tmp = np.load(pathToDataDir+filename)
x_train,x_test,y_train,y_test = tmp['arr_0'],tmp['arr_1'],tmp['arr_2'],tmp['arr_3']

In [12]:
x_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.67433871,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.42319741, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
x_test

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.08871912,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.47627315, ...,  0.        ,
         0.        ,  0.        ]])

In [14]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Try things

Use [OneVsRest](http://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) to perform multi-label (as in [here](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5),...)

**Notes:**
- the tags are heavily imbalanced -> predicts 0 if we do nothing for it

In [16]:
ovr = OneVsRestClassifier(LinearSVC(penalty='l2',
                                    loss='squared_hinge',
                                    C=10, # 0.1, #1.0,
                                    fit_intercept=True,
                                    class_weight='balanced',
                                    random_state=8))

ovr.fit(x_train,y_train)

OneVsRestClassifier(estimator=LinearSVC(C=10, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=8, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [17]:
i = 100000
x_ = x_test[:i,:]
y_ = y_test[:i,:]

y_hat = ovr.predict(x_)

In [20]:
y_hat

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 1, 1]])

In [27]:
y_

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

*Some [metrics](https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics):*

In [28]:
print(avg_jacard(y_,y_hat))

0.130847411217


In [29]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
    tmp_a = None
    if len(set_true) == 0 and len(set_pred) == 0:
        tmp_a = 1
    else:
        tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [30]:
print(hamming_score(y_,y_hat))

0.142857142857


## Decrypt

In [ ]:
vocab_tag = pd.read_csv(pathToDataDir+'vocab_tag.csv',index_col=[0])
vocab_tag

In [ ]:
k = 8

tag = []
for i,b in enumerate(y_[k,:]):
    if b:
        tag.append(vocab_tag.loc[i,'token'])
tag

In [ ]:
# for category in categories:
#     print('... Processing {}'.format(category))
#     # train the model using X_dtm & y
#     ovr.fit(X_train, train[category])
#     # compute the testing accuracy
#     prediction = SVC_pipeline.predict(X_test)
#     print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))